### STE Detector

In [1]:
import numpy as np
import mne
import matplotlib.pyplot as plt
import pandas as pd
from HFODetector import ste
from pathlib import Path

In [7]:
data_root = Path('../hfo/ds003555/derivatives/')
edf_files = sorted(data_root.rglob("*.edf"))

In [3]:

# edf = mne.io.read_raw_edf('../data/ds004100/')
# header = ','.join(edf.ch_names)
# np.savetxt('test1.csv', edf.get_data().T, delimiter=',', header=header)

In [11]:


# edf_path = '../data/ds
detector = ste.STEDetector(
    # filter paramters
    sample_freq=2000.0,  # changed from 500 to 2000
    filter_freq=[80, 200],
    # STE parameters
    rms_window=3*1e-3, 
    min_window=6*1e-3, 
    min_gap=10 * 1e-3, 
    epoch_len=600, 
    min_osc=6, 
    rms_thres=5, 
    peak_thres=3,
    # multi-processing parameters
    n_jobs=4, #Caution: this depends on how many core you CPU has
    front_num=1
)

results = []

for edf_path in edf_files:
    try:
        print(f"Processing: {edf_path.name}")
        
        # Locate matching channels.tsv
        channels_path = edf_path.with_name(edf_path.name.replace("_ieeg.edf", "_channels.tsv"))
        if not channels_path.exists():
            print(f"Missing channel file: {channels_path.name}")
            continue

        # Run HFO detection
        channel_names, start_end = detector.detect_edf(str(edf_path))

        # Filter by good channels
        for ch, intervals in zip(channel_names, start_end):
            hfo_count = len(intervals)
            hfo_duration = sum(e - s for s, e in intervals)
            results.append({
                "file": edf_path.name,
                "channel": ch,
                "n_hfos": hfo_count,
                "duration_samples": hfo_duration,  # divide by 500.0 for seconds if needed
                "duration_sec": hfo_duration / 500.0
            })

    except Exception as e:
        print(f"Error with {edf_path.name}: {e}")

results_df = pd.DataFrame(results)
results_df.to_csv("../hfo/hfo_detection_results.csv")

Processing: sub-01_ses-01_task-hfo_run-01_eeg.edf
Extracting EDF parameters from /Users/billhuang/Desktop/BU_RISE/BU_RISE_Project/hfo/ds003555/derivatives/sub-01/ses-01/eeg/sub-01_ses-01_task-hfo_run-01_eeg.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...


100%|██████████| 51.0/51.0 [00:05<00:00, 9.87it/s]

Processing: sub-01_ses-01_task-hfo_run-02_eeg.edf
Extracting EDF parameters from /Users/billhuang/Desktop/BU_RISE/BU_RISE_Project/hfo/ds003555/derivatives/sub-01/ses-01/eeg/sub-01_ses-01_task-hfo_run-02_eeg.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...



100%|██████████| 51.0/51.0 [00:05<00:00, 10.0it/s]


Processing: sub-01_ses-01_task-hfo_run-03_eeg.edf
Extracting EDF parameters from /Users/billhuang/Desktop/BU_RISE/BU_RISE_Project/hfo/ds003555/derivatives/sub-01/ses-01/eeg/sub-01_ses-01_task-hfo_run-03_eeg.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...


KeyboardInterrupt: 

In [12]:

results_df = pd.DataFrame(results)
results_df.head()

,file,channel,n_hfos,duration_samples,duration_sec
0,sub-01_ses-01_task-hfo_run-01_eeg.edf,Fp1-Fp2,0,0,0.000
1,sub-01_ses-01_task-hfo_run-01_eeg.edf,Fp1-F7,2,200,0.400
2,sub-01_ses-01_task-hfo_run-01_eeg.edf,F7-T3,3,279,0.558
3,sub-01_ses-01_task-hfo_run-01_eeg.edf,T3-T5,0,0,0.000
4,sub-01_ses-01_task-hfo_run-01_eeg.edf,T5-O1,0,0,0.000


In [ ]:
# Need to remove bad channels and use only ictal

### MNI Detector

In [10]:
# channel_names is a list that is the same length as the number of channels in the edf
# start_end is a nested list with the same length as channel_names. start_end[i][j][0] and start_end[i][j][1] 
# will give the start and end index respectively for the jth detected HFO in channel 
print(len(channel_names), len(start_end))

59 59


In [11]:
# convert to pandas dataframe
channel_names = np.concatenate([[channel_names[i]]*len(start_end[i]) for i in range(len(channel_names))])
#removes channels that have no HFO detections
start_end = [start_end[i] for i in range(len(start_end)) if len(start_end[i])>0]
#combines all detected start-end time intervals into one big list
start_end = np.concatenate(start_end)

#Creates a table (DataFrame) with three columns:
# "channel": channel name where the HFO was detected.
#"start": start time (or sample index) of the detected HFO.
#"end": end time (or sample index) of the detected HFO.
HFO_ste_df = pd.DataFrame({"channel":channel_names,"start":start_end[:,0],"end":start_end[:,1]})

In [12]:
HFO_ste_df.head(10)
#channel = which EEG channel detected the HFO
#start = when it started
#end = when it ended

,channel,start,end
0,C3,91126,91156
1,C3,91452,91465
2,C3,97340,97354
3,C4,0,19
4,C4,188898,188937
5,CZ,188907,188932
6,EKG1,86905,86931
7,EKG1,88386,88411
8,EKG2,86921,86942
9,EKG2,88387,88404


In [13]:
HFO_ste_df.to_csv("HFO_ste.csv", index=False)
print(HFO_ste_df.head())


  channel   start     end
0      C3   91126   91156
1      C3   91452   91465
2      C3   97340   97354
3      C4       0      19
4      C4  188898  188937


MNI Detector

In [ ]:


edf_path = 'C:\\Users\\washi\\OneDrive\\Documents\\Final Project\\sub-HUP060\\ses-presurgery\\ieeg\\sub-HUP060_ses-presurgery_task-ictal_acq-seeg_run-01_ieeg.edf' #change this to your edf path
sample_freq=2000 #change this to your sample frequency
detector = mni.MNIDetector(sample_freq, filter_freq=[80, 500], 
            epoch_time=10, epo_CHF=60, per_CHF=95/100, 
            min_win=10*1e-3, min_gap=10*1e-3, thrd_perc=99.9999/100, 
            base_seg=125*1e-3, base_shift=0.5, base_thrd=0.67, base_min=5,
            n_jobs=32, front_num=1)
channel_names, start_end = detector.detect_edf(edf_path)
channel_names = np.concatenate([[channel_names[i]]*len(start_end[i]) for i in range(len(channel_names))])
start_end = [start_end[i] for i in range(len(start_end)) if len(start_end[i])>0]
start_end = np.concatenate(start_end)
HFO_mni_df = pd.DataFrame({"channel":channel_names,"start":start_end[:,0],"end":start_end[:,1]})

Extracting EDF parameters from C:\Users\washi\OneDrive\Documents\Final Project\sub-HUP060\ses-presurgery\ieeg\sub-HUP060_ses-presurgery_task-ictal_acq-seeg_run-01_ieeg.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...


100%|██████████| 58.0/58.0 [01:31<00:00, 1.58s/it]


In [16]:
HFO_mni_df.head(10)
HFO_mni_df.to_csv("HFO_mni.csv", index=False)
print(HFO_mni_df.head())

  channel   start     end
0      C3       0      32
1      C3  181625  189000
2      C4       0      43
3      C4  177875  189000
4      CZ       0      56
